In [1]:
import numpy as np
from sklearn.metrics import accuracy_score

In [21]:
y_test = np.load('../y_test_ratio=0.2.npy')
predict = np.load('resnet.npy')

for i in range(len(y_test)):
    if y_test[i] > 300:
        y_test[i] = 30
    else:
        y_test[i] = int(y_test[i]//10)

y_test = y_test.astype(int)
# print(y_test)

onehot_y_test = []
for i in range(len(y_test)):
    out = [0 for i in range(31)]
    out[y_test[i]]=1
#     print(out)
    onehot_y_test.append(out)
print(predict)

onehot_y_test = np.asarray(onehot_y_test)
print(onehot_y_test)
print(y_test)

[[6.3386190e-01 4.2424705e-05 1.0432019e-03 ... 2.1658142e-03
  6.5607760e-06 3.2513094e-06]
 [3.8668126e-01 3.0958243e-03 7.5835222e-03 ... 4.1894428e-02
  3.6412239e-04 1.5611345e-03]
 [4.0370411e-05 2.7008736e-15 8.8888108e-10 ... 3.7135811e-13
  6.0500069e-15 9.7340411e-17]
 ...
 [2.6513466e-01 1.7837122e-03 3.0198579e-03 ... 1.4123477e-01
  4.7441846e-04 1.0136619e-03]
 [1.9445348e-03 1.6443161e-09 1.6043971e-06 ... 7.3306290e-07
  2.6072411e-09 2.8853459e-10]
 [4.5868955e-04 2.4138168e-13 2.8743912e-08 ... 1.5628374e-10
  8.1847621e-14 3.5778523e-15]]
[[1 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
[ 0  5 27 24 30 10 22  5 15 16 25  8 16  6 10  4 17 12 25 18 30 30 28  6
  8  3 12  0 12 15 14 12  8  5 11 15 30  1  6 30 26 29 16 25 29 30  3  2
  9 13  0  9 18  0 18 19 10 23 22  0 21 29  5 30 25 27 23 20 22  8 24  0
 19 16  4 22 18 12 12  0 12 30  5  0  1  8  8  7  8  8 22  1 17 17 19 22
 10 22  4  6 18  8  7  1

In [22]:
import classic_regression as cr
import keras
import keras.applications.resnet50
from keras.applications.resnet50 import ResNet50
import os
import numpy as np

import os
import tensorflow as tf
import keras.backend.tensorflow_backend as KTF


def get_learning_rate(cnn_type):
    if cnn_type == 'VGG16' or cnn_type == 'VGG16_DROPOUT':
        return 0.00004
    elif cnn_type == 'VGG16_KERAS':
        return 0.00005
    elif cnn_type == 'VGG19':
        return 0.00003
    elif cnn_type == 'VGG19_KERAS':
        return 0.00005
    elif cnn_type == 'RESNET50':
        return 0.00004
    elif cnn_type == 'INCEPTION_V3':
        return 0.00003
    elif cnn_type == 'SQUEEZE_NET':
        return 0.00003
    elif cnn_type == 'DENSENET_161':
        return 0.00003
    elif cnn_type == 'DENSENET_121':
        return 0.00001
    else:
        print('Error Unknown CNN type for learning rate!!')
        exit()
    return 0.00005


def get_optim(cnn_type, optim_type, learning_rate=-1):
    from keras.optimizers import SGD
    from keras.optimizers import Adam

    if learning_rate == -1:
        lr = get_learning_rate(cnn_type)
    else:
        lr = learning_rate
    if optim_type == 'Adam':
        optim = Adam(lr=lr)
    else:
        optim = SGD(lr=lr, decay=1e-6, momentum=0.9, nesterov=True)
    print('Using {} optimizer. Learning rate: {}'.format(optim_type, lr))
    return optim

X_train, y_train, X_test, y_test = cr.load_data()

X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.2, random_state=42)



for i in range(len(y_train)):
    if y_train[i] > 300:
        y_train[i] = 30
    else:
        y_train[i] = int(y_train[i]//10)
        
for i in range(len(y_val)):
    if y_val[i] > 300:
        y_val[i] = 30
    else:
        y_val[i] = int(y_val[i]//10)


for i in range(len(y_test)):
    if y_test[i] > 300:
        y_test[i] = 30
    else:
        y_test[i] = int(y_test[i]//10)

y_test = y_test.astype(int)
y_train = y_train.astype(int)
y_val = y_val.astype(int)
# print(y_test)

onehot_y_train = []
onehot_y_test = []
onehot_y_val = []

for i in range(len(y_test)):
    out = [0 for i in range(31)]
    out[y_test[i]]=1
    print(out)
    onehot_y_test.append(out)

for i in range(len(y_train)):
    out = [0 for i in range(31)]
    out[y_train[i]]=1
    print(out)
    onehot_y_train.append(out)

for i in range(len(y_val)):
    out = [0 for i in range(31)]
    out[y_val[i]]=1
    print(out)
    onehot_y_val.append(out)

print(onehot_y_train)    



dirs = "classification_result"
if not os.path.exists(dirs):
        os.makedirs(dirs)

print(X_train[0].shape)

base_model = ResNet50(include_top= False ,weights='imagenet', input_shape = X_train[0].shape)
x = base_model.output
os.environ["CUDA_VISIBLE_DEVICES"]="0"
# config = tf.ConfigProto()
# config.gpu_options.per_process_gpu_memory_fraction = 0.6 # 每个GPU现存上届控制在60%以内
# session = tf.Session(config=config)

# 设置session
# KTF.set_session(session )

x = keras.layers.GlobalAveragePooling2D()(x)
x = keras.layers.Dropout(0.7)(x)
predictions = keras.layers.Dense(31, activation= 'softmax')(x)

model = keras.models.Model(inputs = base_model.input, outputs = predictions)



optim = get_optim('RESNET50', 'Adam', -1)
model.compile(optimizer=optim, loss='categorical_crossentropy', metrics=['accuracy','val_acc'])

print(X_train)

onehot_y_train = np.asarray(onehot_y_train)
model.fit(X_train,onehot_y_train,epochs=200, batch_size = 8,validation_data=(X_val,y_val))
# score = model.score(X_test, y_test)
result = model.predict(X_test)

np.save(dirs+"/resnet123.npy",result)
# rmse = sqrt(mean_squared_error(y_test,result))

# plt.figure()

# plt.plot(np.arange(len(result)), y_test,'g',label='true value')
# plt.plot(np.arange(len(result)),result,'r',label='predict value')

# plt.title("ResNet50" + ' rmse: %f' % rmse)
# plt.legend()
# plt.savefig(dirs+"/"+"ResNet50"+".png",dpi=300)

Using TensorFlow backend.
/home/dltdc/.local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/dltdc/.local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/dltdc/.local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/dltdc/.local/lib/python3.7/site-packages/tensorflow/python/fra

FileNotFoundError: [Errno 2] No such file or directory: 'X_train_ratio=0.2.npy'

In [18]:
for index,x in enumerate(predict):
    k = [0 for i in range(len(x))]
    k[np.argmax(x)]=1
    predict[index]=k
for x in predict:
    print(x)

score = accuracy_score(onehot_y_test,predict,normalize=True)
score_n = accuracy_score(onehot_y_test,predict,normalize=False)
print(score)
print(score_n)

[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 